In [1]:
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

import time, sys
import datetime
import tqdm.notebook as tqdm
import pandas as pd
import numpy as np

import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

In [3]:
browser = webdriver.Chrome(r"C:\Users\jishn\OneDrive\Desktop\Flight-Price-Prediction\chromedriver.exe")

url = "https://www.expedia.co.in/Flights"
#browser = webdriver.Chrome(r"C:\Users\jishn\OneDrive\Desktop\Flight-Price-Prediction\chromedriver.exe")
# Setting the ticket types paths
browser.get(url)
return_ticket = "//label[@id='flight-type-roundtrip-label-hp-flight']"
one_way_ticket = "//label[@id='flight-type-one-way-label-hp-flight']"
multi_ticket = "//label[@id='flight-type-multi-dest-label-hp-flight']"

In [4]:
def ticket_chooser(ticket):

    try:
        ticket_type = browser.find_element_by_xpath(ticket)
        ticket_type.click()
    except Exception as e:
        pass

In [6]:
# For departure place name
def dep_place_chooser(dep_place):
    fly_from = browser.find_element_by_xpath("//input[@id='uitk-fake-input uitk-form-field-trigger']")
    time.sleep(1)
    fly_from.clear()
    time.sleep(1.5)
    fly_from.send_keys('  ' + dep_place)
    time.sleep(1.5)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1.5)
    first_item.click()

In [7]:
# For arrival place name
def arr_place_chooser(arr_place):
    fly_to = browser.find_element_by_xpath("//input[@id='uitk-fake-input uitk-form-field-trigger']")
    time.sleep(1)
    fly_to.clear()
    time.sleep(1.5)
    fly_to.send_keys('  ' + arr_place)
    time.sleep(1.5)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1.5)
    first_item.click()

In [9]:
# Departure date
def dep_date_chooser(month, day, year):
    dep_date_button = browser.find_element_by_xpath("//input[@id='flight-departing-hp-flight']")
    dep_date_button.clear()
    dep_date_button.send_keys(month + '/' + day + '/' + year)

In [8]:
# Return date
def return_date_chooser(month, day, year):
    return_date_button = browser.find_element_by_xpath("//input[@id='flight-returning-hp-flight']")

    for i in range(11):
        return_date_button.send_keys(Keys.BACKSPACE)
    return_date_button.send_keys(month + '/' + day + '/' + year)

In [11]:
def search():
    search = browser.find_element_by_xpath("//button[@class='uitk-button uitk-button-large uitk-button-fullWidth uitk-button-has-text uitk-button-primary']")
    search.click()
    time.sleep(15)
    print('Results ready!')

In [12]:
df = pd.DataFrame()
def compile_data():
    global df
    global dep_times_list
    global arr_times_list
    global airlines_list
    global price_list
    global durations_list
    global stops_list
    global layovers_list

    #departure times
    dep_times = browser.find_elements_by_xpath("//span[@data-test-id='departure-time']")
    dep_times_list = [value.text for value in dep_times]

    #arrival times
    arr_times = browser.find_elements_by_xpath("//span[@data-test-id='arrival-time']")
    arr_times_list = [value.text for value in arr_times]

    #airline name
    airlines = browser.find_elements_by_xpath("//span[@data-test-id='airline-name']")
    airlines_list = [value.text for value in airlines]

    #prices
    prices = browser.find_elements_by_xpath("//span[@data-test-id='listing-price-dollars']")
    price_list = [value.text.split('/div>)[1] for value in prices]

    #durations
    durations = browser.find_elements_by_xpath("//span[@data-test-id='duration']")
    durations_list = [value.text for value in durations]

    #stops
    stops = browser.find_elements_by_xpath("//span[@class='number-stops']")
    stops_list = [value.text for value in stops]

    #layovers
    layovers = browser.find_elements_by_xpath("//span[@data-test-id='layover-airport-stops']")
    layovers_list = [value.text for value in layovers]

    now = datetime.datetime.now()
    current_date = (str(now.year) + '-' + str(now.month) + '-' + str(now.day))
    current_time = (str(now.hour) + ':' + str(now.minute))
    current_price = 'price' + '(' + current_date + '---' + current_time + ')'
    for i in range(len(dep_times_list)):
        try:
            df.loc[i, 'departure_time'] = dep_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'arrival_time'] = arr_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'airline'] = airlines_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'duration'] = durations_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'stops'] = stops_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'layovers'] = layovers_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, str(current_price)] = price_list[i]
        except Exception as e:
            pass

    print('Comma Separated Value File Created!')

SyntaxError: EOL while scanning string literal (<ipython-input-12-25bfbd68612e>, line 26)

In [13]:
for i in range(8):    
    browser.get(url)
    time.sleep(5)

    # choose flights only
    flights_only = browser.find_element_by_xpath("//button[@id='tab-flight-tab-hp']")
    flights_only.click()

    ticket_chooser(return_ticket)

    dep_place_chooser('BOM')

    arr_place_chooser('BLR')

    dep_date_chooser('04', '01', '2022')

    return_date_chooser('05', '02', '2022')

    search()

    compile_data()

    print('run {} completed!'.format(i))

    df.to_csv('Flight_Price_Data.csv')

    time.sleep(15)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@id='tab-flight-tab-hp']"}
  (Session info: chrome=103.0.5060.66)
Stacktrace:
Backtrace:
	Ordinal0 [0x0088D953+2414931]
	Ordinal0 [0x0081F5E1+1963489]
	Ordinal0 [0x0070C6B8+837304]
	Ordinal0 [0x00739500+1021184]
	Ordinal0 [0x0073979B+1021851]
	Ordinal0 [0x00766502+1205506]
	Ordinal0 [0x007544E4+1131748]
	Ordinal0 [0x00764812+1198098]
	Ordinal0 [0x007542B6+1131190]
	Ordinal0 [0x0072E860+976992]
	Ordinal0 [0x0072F756+980822]
	GetHandleVerifier [0x00AFCC62+2510274]
	GetHandleVerifier [0x00AEF760+2455744]
	GetHandleVerifier [0x0091EABA+551962]
	GetHandleVerifier [0x0091D916+547446]
	Ordinal0 [0x00825F3B+1990459]
	Ordinal0 [0x0082A898+2009240]
	Ordinal0 [0x0082A985+2009477]
	Ordinal0 [0x00833AD1+2046673]
	BaseThreadInitThunk [0x762FFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77497A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77497A6E+238]
